In [ ]:
import sys
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
import os
import inspect
import pandas as pd

project_path = os.path.abspath( os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) + "/../" + "/../")
sys.path.insert(0, project_path)
print(project_path)

from DatabaseStorage.Program.Objects.Human import *
from DatabaseStorage.Program.Objects.Issue import *
from DatabaseStorage.Program.Objects.Machine import *
from DatabaseStorage.Program.Objects.MaintenanceWorkOrder import *
from DatabaseStorage.Program.Objects.Tag import *

from DatabaseStorage.Program.Database.Database_Properties import *
from DatabaseStorage.Program.Database.Database import *
from DatabaseStorage.Program.Database.Main import *

from DatabaseStorage.Program.KPI.KPI import *


from DatabaseStorage.Program.Others.MyDate import *

# DATABASE

## FILES

### NIST server

### Greystone

In [ ]:
PassWord = 'GREYSTONE!!'
#file = os.path.join(project_path, 'DatabaseStorage', 'Data_CSV', 'GreyStone_Data.csv')
file = os.path.join(project_path ,'DatabaseStorage', 'Data_CSV', 'GS_tokenized.csv')
localization = {NodeHuman.VALUE_TECHNICIAN.value: 15,
                NodeHuman.VALUE_OPERATOR.value: 13,

                NodeTag.VALUE_ITEM.value: 17,
                NodeTag.VALUE_PROBLEM.value: 19,
                NodeTag.VALUE_SOLUTION.value: 20,
                NodeTag.VALUE_NA.value: 18,
                NodeTag.VALUE_UNKNOWN.value: 21,
                NodeTag.VALUE_STOP_WORDS.value: 22,

                NodeMachine.VALUE_MACHINE.value: 7,
                NodeIssue.VALUE_MACHINE_DOWN.value: 8,

                NodeIssue.VALUE_DESCRIPTION_PROBLEM.value: 0,
                NodeIssue.VALUE_DESCRIPTION_SOLUTION.value: 1,
                NodeIssue.VALUE_PART_PROCESS.value: 16,
                
                NodeIssue.VALUE_DATE_MAINTENANCE_WORK_ORDER_ISSUE.value: 9,
                NodeIssue.VALUE_DATE_MAINTENANCE_WORK_ORDER_ISSUE.value + "2": 10,
                NodeIssue.VALUE_DATE_MAINTENANCE_WORK_ORDER_CLOSE.value: 11,
                NodeIssue.VALUE_DATE_MAINTENANCE_WORK_ORDER_CLOSE.value + "2": 12
                }

def date_cleanizer(date, time="0000"):
    """
    clean the date from GS data

    :param date: the date
    :param time: the time
    :return: a DateTime
    """

####    DATE ####
    if date is "":
        return None

    date = date.replace(" ","").split("/")

    date[2] = "20" + date[2]

    if len(date[0]) is 1:
        date[0] = "0" + date[0]
    else:
        date[0] = date[0]

    if len(date[1]) is 1:
        date[1] = "0" + date[1]
    else:
        date[1] = date[1]

#### TIME ####
    while len(time) < 4:
        time = "0" + time.replace(" ", "")

    return datetime(year=int(date[2]), month=int(date[0]), day=int(date[1]), hour=int(time[:2]), minute=int(time[2:]))

### Psu

In [ ]:
PassWord = 'PSU!!'
file = os.path.join(project_path, 'DatabaseStorage', 'Data_CSV', 'Psu_Data.csv')
localization = {NodeHuman.VALUE_TECHNICIAN.value: 18,

                NodeMachine.VALUE_MACHINE.value: 3,
                NodeMachine.VALUE_TYPE.value:6,
                NodeMachine.VALUE_MANUFACTURER.value:5,
                NodeMachine.VALUE_LOCASION.value:4,


                NodeIssue.VALUE_DESCRIPTION_PROBLEM.value: 11,
                NodeIssue.VALUE_DESCRIPTION_SOLUTION.value: 19,
                NodeIssue.VALUE_DESCRIPTION_CAUSE.value:12,
                NodeIssue.VALUE_MACHINE_DOWN.value: 13,
                }

### Hvac

In [ ]:
PassWord = 'HVAC!!'
file = os.path.join(project_path, 'DatabaseStorage' , 'Data_CSV', 'Hvac_Data.csv')
localization = {NodeHuman.VALUE_TECHNICIAN.value: 10,
                NodeHuman.VALUE_OPERATOR.value: 43,

                NodeMachine.VALUE_MACHINE.value: 7,

                NodeIssue.VALUE_DESCRIPTION_PROBLEM.value: 5,
                NodeIssue.VALUE_DESCRIPTION_SOLUTION.value: 247,
                }
date_cleanizer = None

### Auto

In [ ]:
PassWord = 'AUTO!!'
file = os.path.join(project_path, 'DatabaseStorage', 'Data_CSV', 'Auto_Data.csv')
localization = {NodeHuman.VALUE_TECHNICIAN.value: 4,
                NodeHuman.VALUE_OPERATOR.value: 11,
                NodeHuman.VALUE_CRAFTS.value: 16,
                NodeHuman.VALUE_SKILLS.value: 17,

                NodeMachine.VALUE_MACHINE.value: 8,

                NodeIssue.VALUE_DESCRIPTION_PROBLEM.value: 5,
                NodeIssue.VALUE_DESCRIPTION_SOLUTION.value: 7,
                }

### Mine

In [ ]:
PassWord = 'MINE!!'
file = os.path.join(project_path, 'DatabaseStorage', 'Data_CSV', 'Mine_Data.csv')

localization = {}

## ACTION

### Connect

In [ ]:
database = DatabaseNeo4J("bolt://127.0.0.1:7687", "neo4j", PassWord)

In [ ]:
database = DatabaseNeo4J("bolt://127.0.0.1:7687", "neo4j", "11/Jul/2017!")

## Indexs

In [ ]:
database.createIndexes()

In [ ]:
database.dropIndexes()

## Data

In [ ]:
graph_database_from_csv(database, file, localization, date_cleanizer)

In [ ]:
# token_dataframe = pd.DataFrame.from_csv("GS_tokenized_forSascha.csv", index_col =None)
# graph_database_from_tag_extractor(database = database, token_dataframe = token_dataframe)

In [ ]:
database.deleteData()

## NEW

# KPI

# KPI using Object

In [ ]:
tec = Technician(name=["h1","h2"])
ope = Operator(name=["_"])

i3=Issue(problem=["_"], solution=["_"], machine_down="_",date_maintenance_work_order_issue=["_"], date_maintenance_work_order_close=["_"],part_in_process=["_"] )
i=Issue(date_maintenance_work_order_issue=["_"])
mac = Machine(name=["_","h2", "h1"])

tagI = TagItem(keyword = ["_"])
tagS = TagAction(keyword = ["_"] , it_is="s")
tagP = TagAction(keyword = ["_"] , it_is="p")



In [ ]:
objects = [i,mac]
query, results = cypher_from_kpi(objects)

In [ ]:
dataframe, results = pandas_from_cypher_kpi(database, query, results)

In [ ]:
print(results)

In [ ]:
dataframe.head()

In [ ]:
df = dataframe
df["issue.date_maintenance_work_order_issue"] = pd.DatetimeIndex(df["issue.date_maintenance_work_order_issue"])
print(type(df["issue.date_maintenance_work_order_issue"][0]))

In [ ]:
g=df.groupby(by=['machine.name',pd.Grouper(key='issue.date_maintenance_work_order_issue', freq="M")]).sum()
g=g.reset_index()
g.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib as mpl


sns.set(style="darkgrid")
fig =plt.figure(figsize=(12,8))

df=g.reset_index()

ax = sns.tsplot(data=df,
          time="issue.date_maintenance_work_order_issue",
          unit="machine.name",
          condition="machine.name",
          value="issue_count"
         )

def myFormatter(x, pos):
    return pd.to_datetime(x).strftime('%b-%Y')

# assign locator and formatter for the xaxis ticks.
ax.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(myFormatter))

# put the labels at 45deg since they tend to be too long
fig.autofmt_xdate()
plt.show()


In [ ]:
dataframe


In [ ]:
y= dataframe[["machine.name", "issue_count"]]


In [ ]:
y